In [4]:
import numpy as np
import cv2 # ? opencv
import matplotlib.pyplot as plt
import skimage.feature
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from PIL import Image

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [54]:
def get_data(filename):
    image_1 = cv2.imread("input/TrainDotted/" + filename)
    image_2 = cv2.imread("input/Train/" + filename)
    img1 = cv2.GaussianBlur(image_1,(5,5),0)

    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 50] = 0
    mask_1[mask_1 > 0] = 255
    image_4 = cv2.bitwise_or(image_3, image_3, mask=mask_1)

    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_6 = np.max(image_4,axis=2)

    # detect blobs
    blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=7, num_sigma=1, threshold=0.02)

    print(blobs.shape[0])
    image_train = cv2.resize(image_2, (300, 200))
    return image_train, blobs.shape[0]

In [56]:
import glob
from os import listdir
from os.path import isfile, join, basename
#get_data("5.png")
trainX = []
trainY = []
for i, filename in enumerate(glob.iglob("input/Train/*.png")):
    X, Y = get_data(basename(filename))
    print(i)
    trainX.append(X)
    trainY.append(Y)
trainX = np.array(trainX)
trainY = np.array(trainY)
# onlyfiles = [f for f in listdir("input/Train/") if isfile(join("input/Train/", f))]
# print(onlyfiles)


4
0
4
1
3
2
5
3
5
4
7
5
2
6
3
7
7
8
2
9
2
10
3
11
5
12
3
13
4
14
6
15
4
16
1
17
0
18
4
19
3
20
1
21
6
22
2
23
2
24
6
25
2
26
2
27
8
28
4
29
2
30
3
31
2
32
1
33
5
34
3
35
1
36
5
37
3
38
3
39
5
40
2
41
1
42
4
43
2
44
1
45
2
46
8
47
2
48
3
49
0
50
3
51
5
52
5
53
3
54
5
55
4
56
2
57
3
58
2
59
4
60
4
61
2
62
5
63
3
64
2
65
7
66
3
67
3
68
8
69
5
70
3
71
4
72
6
73
3
74
6
75
2
76
1
77
4
78
5
79
5
80
3
81
4
82
2
83
2
84
5
85
2
86
3
87
5
88
1
89
5
90


In [57]:
trainX.shape

(91, 200, 300, 3)

In [61]:
'''
In this script, we provide a simple yet powerful function that uses 
image augmentation techniques.
This helps to deal with less number of training instances, 
increase accuracy, generalize models, etc.

Mandatory packages to be installed:
    tensorflow
    numpy
    
You can save this script in a separate file and import the function in your main file(s).

If you find this script useful, please upvote. Thanks
'''

import tensorflow as tf


'''
    This function peforms various data augmentation techniques to the dataset
    
    @parameters:
        dataset: the feature training dataset in numpy array with shape [num_examples, num_rows, num_cols, num_channels] (since it is an image in numpy array)
        dataset_labels: the corresponding training labels of the feature training dataset in the same order, and numpy array with shape [num_examples, <anything>]
        augmentation_factor: how many times to perform augmentation.
        use_random_rotation: whether to use random rotation. default: true
        use_random_shift: whether to use random shift. default: true
        use_random_shear: whether to use random shear. default: true
        use_random_zoom: whether to use random zoom. default: true
        
    @returns:
        augmented_image: augmented dataset
        augmented_image_labels: labels corresponding to augmented dataset in order.
        
    for the augmentation techniques documentation, go here:
    	https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_rotation
    	https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_shear
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_shift
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_zoom
'''
def augment_data(dataset, dataset_labels, augementation_factor=1, use_random_rotation=False, use_random_shear=False, use_random_shift=False, use_random_zoom=False):
	augmented_image = []
	augmented_image_labels = []
	zoom_tuple = (0.8, 0.9)

	for num in range (0, dataset.shape[0]):

		for i in range(0, augementation_factor):
			# original image:
			augmented_image.append(dataset[num])
			augmented_image_labels.append(dataset_labels[num])

			if use_random_rotation:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_rotation(dataset[num], 20, row_axis=0, col_axis=1, channel_axis=2))
				augmented_image_labels.append(dataset_labels[num])

			if use_random_shear:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_shear(dataset[num], 0.2, row_axis=0, col_axis=1, channel_axis=2))
				augmented_image_labels.append(dataset_labels[num])

			if use_random_shift:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_shift(dataset[num], 0.2, 0.2, row_axis=0, col_axis=1, channel_axis=2))
				augmented_image_labels.append(dataset_labels[num])

			if use_random_zoom:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_zoom(dataset[num], zoom_tuple, row_axis=0, col_axis=1, channel_axis=2))
				augmented_image_labels.append(dataset_labels[num])

	return np.array(augmented_image), np.array(augmented_image_labels)
	
'''
If you find this script useful, please upvote. Thanks
'''

'\nIf you find this script useful, please upvote. Thanks\n'

In [62]:
trainX_plus, trainY_plus = augment_data(trainX, trainY, augementation_factor=1)

In [63]:
trainX_plus.shape

(91, 200, 300, 3)

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX_plus, trainY_plus, test_size=0.1, random_state=7)

In [65]:
from keras import Model

In [66]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False, input_shape=(200,300,3))
last = model.output
x = Flatten()(last)
x = Dense(256)(x)
x = LeakyReLU(alpha=.1)(x)
preds = Dense(1, activation='linear')(x)
model = Model(model.input, preds)

In [11]:
optim = keras.optimizers.SGD(lr=1e-5, momentum=0.2)

In [67]:
optim = keras.optimizers.SGD(lr=1e-5, momentum=0.2)
model.compile(loss='mean_squared_error', optimizer=optim)
model.fit(trainX_plus, trainY_plus, batch_size=36, epochs=100, verbose=2, validation_split=0.2)

Train on 72 samples, validate on 19 samples
Epoch 1/100
 - 5s - loss: 23.0344 - val_loss: 3.7648
Epoch 2/100
 - 3s - loss: 3.0162 - val_loss: 3.6684
Epoch 3/100
 - 3s - loss: 2.8148 - val_loss: 3.7602
Epoch 4/100
 - 3s - loss: 2.6199 - val_loss: 3.7493
Epoch 5/100
 - 3s - loss: 2.2894 - val_loss: 3.8099
Epoch 6/100
 - 3s - loss: 2.1324 - val_loss: 3.8291
Epoch 7/100
 - 3s - loss: 1.9472 - val_loss: 3.8958
Epoch 8/100
 - 3s - loss: 1.7972 - val_loss: 3.8869
Epoch 9/100
 - 3s - loss: 1.6805 - val_loss: 3.8878
Epoch 10/100
 - 3s - loss: 1.5739 - val_loss: 4.0056
Epoch 11/100
 - 3s - loss: 1.4339 - val_loss: 4.0082
Epoch 12/100
 - 3s - loss: 1.3813 - val_loss: 4.0818
Epoch 13/100
 - 3s - loss: 1.2387 - val_loss: 4.0371
Epoch 14/100
 - 3s - loss: 1.1560 - val_loss: 4.0370
Epoch 15/100
 - 3s - loss: 1.0705 - val_loss: 4.0733
Epoch 16/100
 - 3s - loss: 1.0242 - val_loss: 4.0628
Epoch 17/100
 - 3s - loss: 0.9455 - val_loss: 4.1527
Epoch 18/100
 - 3s - loss: 0.8757 - val_loss: 4.1487
Epoch 19/1

In [68]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [69]:
result = model.predict(X_train)
print('Training set --')
print('    ground truth: ', y_train)
print('  evaluate count: ', np.ndarray.round(result*(result>0.5)))

result_test = model.predict(X_test)
print('Testing set --')
print('    ground truth: ', y_test)
print('   predict count: ', np.ndarray.round(result_test*(result_test>0.5)))

Training set --
    ground truth:  [3 3 1 5 1 3 3 2 3 1 5 7 2 5 2 5 1 5 3 3 3 1 2 2 4 5 1 0 2 3 6 8 4 3 4 8 4
 5 5 5 3 4 2 2 5 3 2 2 7 5 6 2 2 5 3 2 4 5 3 4 4 2 5 4 2 3 2 1 3 7 1 4 5 4
 2 2 2 3 6 3 8]
  evaluate count:  [[3.]
 [3.]
 [3.]
 [5.]
 [1.]
 [3.]
 [3.]
 [2.]
 [3.]
 [1.]
 [5.]
 [7.]
 [2.]
 [4.]
 [2.]
 [3.]
 [1.]
 [4.]
 [3.]
 [3.]
 [3.]
 [1.]
 [2.]
 [2.]
 [1.]
 [5.]
 [1.]
 [0.]
 [2.]
 [3.]
 [3.]
 [8.]
 [4.]
 [3.]
 [4.]
 [8.]
 [4.]
 [5.]
 [2.]
 [5.]
 [3.]
 [4.]
 [4.]
 [2.]
 [5.]
 [3.]
 [5.]
 [2.]
 [7.]
 [5.]
 [3.]
 [2.]
 [3.]
 [5.]
 [3.]
 [2.]
 [4.]
 [5.]
 [2.]
 [4.]
 [4.]
 [2.]
 [5.]
 [4.]
 [2.]
 [3.]
 [2.]
 [5.]
 [3.]
 [7.]
 [1.]
 [2.]
 [3.]
 [4.]
 [2.]
 [2.]
 [4.]
 [3.]
 [6.]
 [3.]
 [8.]]
Testing set --
    ground truth:  [0 6 3 3 5 6 2 4 2 4]
   predict count:  [[-0.]
 [ 6.]
 [ 3.]
 [ 3.]
 [ 5.]
 [ 6.]
 [ 2.]
 [ 4.]
 [ 2.]
 [ 4.]]


In [70]:
print("train", "prediction", "difference")
for i in range(result.shape[0]):
    print(y_train[i], round(result[i][0]), round(result[i][0]) - y_train[i])

train prediction difference
3 3.0 0.0
3 3.0 0.0
1 3.0 2.0
5 5.0 0.0
1 1.0 0.0
3 3.0 0.0
3 3.0 0.0
2 2.0 0.0
3 3.0 0.0
1 1.0 0.0
5 5.0 0.0
7 7.0 0.0
2 2.0 0.0
5 4.0 -1.0
2 2.0 0.0
5 3.0 -2.0
1 1.0 0.0
5 4.0 -1.0
3 3.0 0.0
3 3.0 0.0
3 3.0 0.0
1 1.0 0.0
2 2.0 0.0
2 2.0 0.0
4 1.0 -3.0
5 5.0 0.0
1 1.0 0.0
0 0.0 0.0
2 2.0 0.0
3 3.0 0.0
6 3.0 -3.0
8 8.0 0.0
4 4.0 0.0
3 3.0 0.0
4 4.0 0.0
8 8.0 0.0
4 4.0 0.0
5 5.0 0.0
5 2.0 -3.0
5 5.0 0.0
3 3.0 0.0
4 4.0 0.0
2 4.0 2.0
2 2.0 0.0
5 5.0 0.0
3 3.0 0.0
2 5.0 3.0
2 2.0 0.0
7 7.0 0.0
5 5.0 0.0
6 3.0 -3.0
2 2.0 0.0
2 3.0 1.0
5 5.0 0.0
3 3.0 0.0
2 2.0 0.0
4 4.0 0.0
5 5.0 0.0
3 2.0 -1.0
4 4.0 0.0
4 4.0 0.0
2 2.0 0.0
5 5.0 0.0
4 4.0 0.0
2 2.0 0.0
3 3.0 0.0
2 2.0 0.0
1 5.0 4.0
3 3.0 0.0
7 7.0 0.0
1 1.0 0.0
4 2.0 -2.0
5 3.0 -2.0
4 4.0 0.0
2 2.0 0.0
2 2.0 0.0
2 4.0 2.0
3 3.0 0.0
6 6.0 0.0
3 3.0 0.0
8 8.0 0.0


In [71]:
print("test", "prediction", "difference")
count = 0
for i in range(result_test.shape[0]):
    print(y_test[i], round(result_test[i][0]), round(result_test[i][0]) - y_test[i])
    if (round(result_test[i][0]) - y_test[i] == 0):
        count += 1
print("Accuracy", count / result_test.shape[0])

test prediction difference
0 -0.0 -0.0
6 6.0 0.0
3 3.0 0.0
3 3.0 0.0
5 5.0 0.0
6 6.0 0.0
2 2.0 0.0
4 4.0 0.0
2 2.0 0.0
4 4.0 0.0
Accuracy 1.0


In [35]:
# import glob
# from os.path import isfile, join, basename
testX = []
testY = [5, 8, 5, 2, 4, 5, 3, 2, 6, 6, 6, 7, 6, 5, 3, 3] #0,10,...,
for filename in glob.iglob("input/Test/*.png"):
    filename = basename(filename)
    print(filename)
    image_test = cv2.imread("input/Test/" + filename)
    image_test = cv2.resize(image_test, (300, 200))
    testX.append(image_test)
testX = np.array(testX)
testY = np.array(testY)

19.png
0.png
4.png
18.png
1.png
20.png
11.png
17.png
2.png
3.png
10.png
16.png
12.png
13.png
14.png
15.png


In [2]:
def get_testdata(filename):
    image_1 = cv2.imread("input/TestDotted/" + filename)
    image_2 = cv2.imread("input/Test/" + filename)
    img1 = cv2.GaussianBlur(image_1,(5,5),0)

    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 50] = 0
    mask_1[mask_1 > 0] = 255
    image_4 = cv2.bitwise_or(image_3, image_3, mask=mask_1)

    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_6 = np.max(image_4,axis=2)

    # detect blobs
    blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=7, num_sigma=1, threshold=0.02)
    print(blobs.shape[0])
    image_train = cv2.resize(image_2, (300, 200))
    return image_train, blobs.shape[0]

In [6]:
import glob
from os import listdir
from os.path import isfile, join, basename
testX = []
testY = []
for i, filename in enumerate(glob.iglob("input/Test/*.png")):
    X, Y = get_testdata(basename(filename))
    print(i)
    testX.append(X)
    testY.append(Y)
testX = np.array(testX)
testY = np.array(testY)

8
0
12
1
5
2
6
3
4
4
9
5
7
6
4
7
6
8
12
9
4
10
8
11
10
12
6
13
6
14
8
15
5
16
9
17
6
18
7
19


In [12]:

# later...
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer=optim)
score = loaded_model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


IndexError: list index out of range

In [13]:
result_test2 = loaded_model.predict(testX)
print('Testing set --')
print('    ground truth: ', testY)
print('   predict count: ', np.ndarray.round(result_test2*(result_test2>0.5)))
print("test", "prediction", "difference")
count = 0
for i in range(result_test2.shape[0]):
    print(testY[i], round(result_test2[i][0]), round(result_test2[i][0]) - testY[i])
    if (round(result_test2[i][0]) - testY[i] == 0):
        count += 1
print("Accuracy", count / result_test2.shape[0])

Testing set --
    ground truth:  [ 8 12  5  6  4  9  7  4  6 12  4  8 10  6  6  8  5  9  6  7]
   predict count:  [[5.]
 [2.]
 [3.]
 [3.]
 [4.]
 [2.]
 [3.]
 [3.]
 [3.]
 [4.]
 [3.]
 [4.]
 [4.]
 [4.]
 [4.]
 [2.]
 [3.]
 [3.]
 [5.]
 [6.]]
test prediction difference
8 5.0 -3.0
12 2.0 -10.0
5 3.0 -2.0
6 3.0 -3.0
4 4.0 0.0
9 2.0 -7.0
7 3.0 -4.0
4 3.0 -1.0
6 3.0 -3.0
12 4.0 -8.0
4 3.0 -1.0
8 4.0 -4.0
10 4.0 -6.0
6 4.0 -2.0
6 4.0 -2.0
8 2.0 -6.0
5 3.0 -2.0
9 3.0 -6.0
6 5.0 -1.0
7 6.0 -1.0
Accuracy 0.05


In [41]:
print(score)
result_test3 = loaded_model.predict(testX)

4.546081066131592


In [42]:
result_test2 = model.predict(testX)
print('Testing set --')
print('    ground truth: ', testY)
print('   predict count: ', np.ndarray.round(result_test3*(result_test3>0.5)))

Testing set --
    ground truth:  [5 8 5 2 4 5 3 2 6 6 6 7 6 5 3 3]
   predict count:  [[5.]
 [6.]
 [3.]
 [7.]
 [6.]
 [4.]
 [6.]
 [5.]
 [5.]
 [4.]
 [7.]
 [5.]
 [5.]
 [4.]
 [5.]
 [5.]]


In [43]:
print("test", "prediction", "difference")
count = 0
for i in range(result_test3.shape[0]):
    print(testY[i], round(result_test3[i][0]), round(result_test3[i][0]) - testY[i])
    if (round(result_test3[i][0]) - testY[i] == 0):
        count += 1
print("Accuracy", count / result_test3.shape[0])

test prediction difference
5 5.0 0.0
8 6.0 -2.0
5 3.0 -2.0
2 7.0 5.0
4 6.0 2.0
5 4.0 -1.0
3 6.0 3.0
2 5.0 3.0
6 5.0 -1.0
6 4.0 -2.0
6 7.0 1.0
7 5.0 -2.0
6 5.0 -1.0
5 4.0 -1.0
3 5.0 2.0
3 5.0 2.0
Accuracy 0.0625
